In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, UpSampling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar100
import numpy as np
import cv2
import os
from google.colab import files
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Định nghĩa hàm tạo mô hình ResNet50 để trích xuất đặc trưng
def build_full_model():
    resnet_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )

    for layer in resnet_model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

    model = Sequential()
    model.add(UpSampling2D(size=(7, 7), interpolation='bilinear', input_shape=(32, 32, 3)))
    model.add(resnet_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(100, activation='softmax'))

    return model

# Xây dựng mô hình chỉ để trích xuất đặc trưng
def build_feature_extractor():
    resnet_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )

    for layer in resnet_model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

    model = Sequential()
    model.add(UpSampling2D(size=(7, 7), interpolation='bilinear', input_shape=(32, 32, 3)))
    model.add(resnet_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))

    return model
# Hàm chuẩn hóa ảnh và trích xuất đặc trưng
def extract_feature(feature_extractor, img):
    img = img * 1.0 / 255
    resized_img = cv2.resize(img, (224, 224))
    resized_img = resized_img.reshape((1, 224, 224, 3))
    predict_img = feature_extractor.predict(resized_img)
    feature = predict_img[0].tolist()
    return feature

# Hàm chuẩn hóa ảnh và trích xuất đặc trưng
def extract_feature(model, img):
    img = img * 1.0 / 255
    resized_img = cv2.resize(img, (32, 32))  # Resize về 32x32 trước khi upsample
    resized_img = resized_img.reshape((1, 32, 32, 3))
    predict_img = model.predict(resized_img)
    return predict_img[0]

In [ ]:
# Tải bộ dữ liệu CIFAR-100
cifar100 = tf.keras.datasets.cifar100
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()



In [ ]:
# Đường dẫn tới tệp trọng số bạn muốn dự đoán
weight_path = '/content/drive/My Drive/weights/transfer_resnet50_cifar100.h5'
# Xây dựng mô hình đầy đủ
full_model = build_full_model()

# Khởi tạo mô hình bằng cách chạy nó với một đầu vào giả lập
dummy_input = np.zeros((1, 32, 32, 3))  # CIFAR-100 đầu vào là (32, 32, 3)
full_model.predict(dummy_input)

# Kiểm tra xem tệp trọng số có tồn tại không
if os.path.exists(weight_path):
    try:
        full_model.load_weights(weight_path)
        print("Đã tải trọng số thành công.")
    except OSError as e:
        print(f"Lỗi khi mở tệp: {e}")
else:
    print("Tệp trọng số không tồn tại.")

1/1 [==============================] - 3s 3s/step
Đã tải trọng số thành công.


In [ ]:
# Xây dựng mô hình chỉ để trích xuất đặc trưng
feature_extractor = build_feature_extractor()

# Sao chép trọng số từ mô hình đầy đủ sang mô hình trích xuất đặc trưng
for i in range(len(feature_extractor.layers)):
    feature_extractor.layers[i].set_weights(full_model.layers[i].get_weights())

In [ ]:
# Trích xuất đặc trưng cho tập huấn luyện và tập kiểm tra
train_features = []
test_features = []
train_labels = []
test_labels = []

In [ ]:
for index in range(len(X_train)):
    feature = extract_feature(feature_extractor, X_train[index])
    label = int(Y_train[index][0])
    train_features.append(feature)
    train_labels.append(label)

# Chia dữ liệu tập huấn luyện thành train và validation
# CIFAR-100 không có tập validation, chúng ta sẽ dùng một phần của tập huấn luyện làm tập validation
split_index = int(len(train_features) * 0.3)
X_train_features_1 = train_features[:split_index]
X_train_features_2 = train_features[split_index:2*split_index]
X_train_features_3 = train_features[2*split_index:]

Y_train_labels_1 = train_labels[:split_index]
Y_train_labels_2 = train_labels[split_index:2*split_index]
Y_train_labels_3 = train_labels[2*split_index:]


with open('/content/drive/My Drive/feature/ind.cifar100_1.x', 'wb') as file:
    pickle.dump(np.array(X_train_features_1), file)

with open('/content/drive/My Drive/feature/ind.cifar100_2.x', 'wb') as file:
    pickle.dump(np.array(X_train_features_2), file)

with open('/content/drive/My Drive/feature/ind.cifar100_3.x', 'wb') as file:
    pickle.dump(np.array(X_train_features_3), file)

with open('/content/drive/My Drive/feature/ind.cifar100_1.y', 'wb') as file:
    pickle.dump(np.array(Y_train_labels_1), file)

with open('/content/drive/My Drive/feature/ind.cifar100_2.y', 'wb') as file:
    pickle.dump(np.array(Y_train_labels_2), file)

with open('/content/drive/My Drive/feature/ind.cifar100_3.y', 'wb') as file:
    pickle.dump(np.array(Y_train_labels_3), file)

In [ ]:
for index in range(len(X_test)):
    feature = extract_feature(feature_extractor, X_test[index])
    label = int(Y_test[index][0])
    test_features.append(feature)
    test_labels.append(label)

split_index_test = int(len(test_features) * 0.5)
X_test_features_1 = test_features[:split_index_test]
X_test_features_2 = test_features[split_index_test:]

Y_test_labels_1 = test_labels[:split_index_test]
Y_test_labels_2 = test_labels[split_index_test:]

with open('/content/drive/My Drive/feature/ind.cifar100_1.tx', 'wb') as file:
    pickle.dump(np.array(X_test_features_1), file)

with open('/content/drive/My Drive/feature/ind.cifar100_2.tx', 'wb') as file:
    pickle.dump(np.array(X_test_features_2), file)

with open('/content/drive/My Drive/feature/ind.cifar100_1.ty', 'wb') as file:
    pickle.dump(np.array(Y_test_labels_1), file)

with open('/content/drive/My Drive/feature/ind.cifar100_2.ty', 'wb') as file:
    pickle.dump(np.array(Y_test_labels_2), file)

In [ ]:
!unzip /content/drive/MyDrive/fake_1.zip -d /content/drive/MyDrive

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/fake_1/15001.png  
  inflating: /content/drive/MyDrive/fake_1/15002.png  
  inflating: /content/drive/MyDrive/fake_1/15003.png  
  inflating: /content/drive/MyDrive/fake_1/15004.png  
  inflating: /content/drive/MyDrive/fake_1/15005.png  
  inflating: /content/drive/MyDrive/fake_1/15006.png  
  inflating: /content/drive/MyDrive/fake_1/15007.png  
  inflating: /content/drive/MyDrive/fake_1/15008.png  
  inflating: /content/drive/MyDrive/fake_1/15009.png  
  inflating: /content/drive/MyDrive/fake_1/15010.png  
  inflating: /content/drive/MyDrive/fake_1/15011.png  
  inflating: /content/drive/MyDrive/fake_1/15012.png  
  inflating: /content/drive/MyDrive/fake_1/15013.png  
  inflating: /content/drive/MyDrive/fake_1/15014.png  
  inflating: /content/drive/MyDrive/fake_1/15015.png  
  inflating: /content/drive/MyDrive/fake_1/15016.png  
  inflating: /content/drive/MyDrive/fake_1/15017.png  
  inflating: /

In [ ]:
# Trích xuất đặc trưng cho ảnh fake
fake_features = []

In [ ]:
# Truy cập vào thư mục fake_img để đọc các ảnh
path = '/content/drive/MyDrive/fake_1'
print(len(os.listdir(path)))
list_img = []
for image in os.listdir(path):
    img = cv2.imread(os.path.join(path, image))
    feature = extract_feature(feature_extractor, img)
    fake_features.append(feature)
    list_img.append(img)
print(len(list_img))

# Lưu lại các đặc trưng vào file
with open('/content/drive/My Drive/feature/ind.fake_1.x', 'wb') as file:
    pickle.dump(np.array(fake_features), file)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 45ms/step
20000
